In [1]:
import data
import torch
from torch.utils.data import TensorDataset
import numpy as np
from scipy.stats import entropy

import gans.w_gan as wgan
import gans.w_gp_gan as wgpgan
import gans.vae as vae
import gans.ns_gan as nsgan
import gans.mm_gan as mmgan
import gans.ls_gan as lsgan
import gans.dra_gan as dragan
import gans.be_gan as began
from gans.load_data import get_data
from gans.utils import to_var

In [2]:
n=1
size=200
i = 0
j = 0
distributions = ['normal', 'beta', 'exponential', 'gamma', 'gumbel', 'laplace']
pdfs = ['uniform', 'random']

In [3]:
def preprocess(generator, samples, BATCH_SIZE = 100):
    data_train = torch.from_numpy(generator.generate_samples(samples)).float()
    data_val = torch.from_numpy(generator.generate_samples(samples)).float()
    data_test = torch.from_numpy(generator.generate_samples(samples)).float()
    labels_train = torch.from_numpy(np.zeros((samples,1)))
    train = TensorDataset(data_train, labels_train)
    val =TensorDataset(data_val, labels_train)
    test = TensorDataset(data_test, labels_train)
    train_iter = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
    val_iter = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=True)
    test_iter = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)
    return train_iter, val_iter, test_iter

# Multivariate Distributions Example

In [4]:
# generator = data.Distribution(distributions[i], 200)
# generator.save_dist('output_file.pickle')  # Saving the generator

# Mixture Models Example

In [5]:
# generator = data.MixtureDistribution(distributions[i], distributions[j], n_mixtures=15, dim=200)
# generator.get_log_likelihood(data_train)
# generator.save_dist()

# Circles Example

In [6]:
# generator = data.CirclesDatasetGenerator(size=256, n_circles=4, random_colors=True, random_sizes=True, modes=20)
# x = generator.generate_samples(10000)
# print(len(x))
# print(x[0].shape)
# generator.generate_samples_to_directory(5000, './dataset_1')
# generator.save_generator('./generator1.pickle')

# MNIST Data

In [7]:
train_iter, val_iter, test_iter = get_data()

In [8]:
train_iter.dataset.data_tensor.shape

torch.Size([50000, 1, 28, 28])

# Preprocessing Synthetic Data

In [9]:
generator = data.Distribution(distributions[i], size) 

In [10]:
train_iter, val_iter, test_iter = preprocess(generator, 10000)

In [11]:
train_iter.dataset.data_tensor.shape

torch.Size([10000, 200])

# Controlled Experiments on GANs

## 1. Divergence Measures between Distributions

#### Measuring divergence between Synthetic Data and GAN Data with the following metrics: KL Divergences, Jensen-Shannon Divergence, Optimal Tranport Distance, Wasserstein Distance, Energy Distance, Total Variation Distance


In [12]:
# Example with WGAN
model = wgan.GAN(image_size = size, hidden_dim = 256, z_dim = 1)
if torch.cuda.is_available():
    model = model.cuda()
trainer = wgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 5e-5, D_lr = 5e-5, D_steps = 5, clip = 0.01)

A Jupyter Widget

Epoch[1/1], G Loss: -0.0055, D Loss: -0.2837



In [13]:
data_actual = torch.from_numpy(generator.generate_samples(10000)).float()
data_actual = to_var(data_actual)
noise = to_var(torch.randn(1000, 1))
data_gan = model.G(noise)
a = data_actual.data.numpy()
b = data_gan.data.numpy()


In [14]:
a

array([[-2.05917382, -0.36331111, -0.83420134, ..., -1.04250824,
        -1.07983208,  0.54676396],
       [-0.11467439, -0.00607845,  0.66130626, ..., -0.98574871,
         0.71506691,  1.52987111],
       [ 0.64632928,  0.44065332,  0.30837309, ...,  1.06818366,
        -0.54909253, -0.05914635],
       ..., 
       [-0.24698199,  1.04276586,  1.87095046, ...,  1.12448835,
        -0.02806244,  1.89680398],
       [-0.66029161, -0.3672798 ,  0.57202709, ..., -0.29143873,
         0.52248794,  1.13417041],
       [ 2.35224724,  1.67268836,  0.89586365, ...,  0.27577114,
         1.11837709, -0.02581001]], dtype=float32)

In [15]:
b

array([[ 0.46247408,  0.54285944,  0.50542468, ...,  0.44964212,
         0.60158163,  0.43732852],
       [ 0.45528194,  0.55819452,  0.53013235, ...,  0.44677815,
         0.5556947 ,  0.47703022],
       [ 0.38111433,  0.5419057 ,  0.54269558, ...,  0.40546203,
         0.49150854,  0.51318818],
       ..., 
       [ 0.46399528,  0.54550892,  0.50571424, ...,  0.45196706,
         0.59539515,  0.44478455],
       [ 0.43420798,  0.54904872,  0.50481588, ...,  0.41821757,
         0.63725603,  0.38978702],
       [ 0.40744522,  0.54773605,  0.54244715, ...,  0.4189299 ,
         0.5098325 ,  0.50150669]], dtype=float32)

In [16]:
# ll_1 = generator.get_log_likelihood(a)
# ll_2 = generator.get_log_likelihood(b)

In [17]:
print(entropy(pk=[0.2, 0.2], qk=[0.75, 0.78]))
print(entropy(pk=[.99, 0.01], qk=[0.49, 0.51]))

0.000192270719539
0.656948300176


## 2. Empirical Analysis of Mode Collapse

#### Three methods to measure mode collapse Inception Score, MODE Score, Frechet Inception Distance

## 3. Empirical Evaluation of Performance with varying Information Geometry

## 4. Evaluation of Empirical Sample Complexity

In [18]:
n = 1

# WGAN

In [19]:
model = wgan.GAN(image_size = size, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = wgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 5e-5, D_lr = 5e-5, D_steps = 5, clip = 0.01)

A Jupyter Widget

Epoch[1/1], G Loss: -0.0024, D Loss: -0.2949



# WGAN GP

In [20]:
model = wgpgan.GAN(image_size = size, hidden_dim = 256, z_dim = 128) 
if torch.cuda.is_available():
    model = model.cuda()
trainer = wgpgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 1e-4, D_lr = 1e-4, D_steps = 5)

A Jupyter Widget

Epoch[1/1], G Loss: -0.1147, D Loss: 0.6302



# VAE

In [21]:
model = vae.VAE(image_size = size, hidden_dim = 400, z_dim = 20)
if torch.cuda.is_available():
    model.cuda()
trainer = vae.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model, num_epochs = n)

# # Explore latent space
# viz = vae.Viz(train_iter, val_iter, test_iter, model)
# viz.make_all()

A Jupyter Widget

Epoch[1/1], Total Loss: -131097.4583, Reconst Loss: -141953.2655, KL Div: 10855.8065222, Val Loss: -170863.9643



# NS GAN

In [22]:
model = nsgan.GAN(image_size = size, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = nsgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 2e-4, D_lr = 2e-4, D_steps = 1)

A Jupyter Widget

Epoch[1/1], G Loss: 1.1754, D Loss: 0.7038



# MM GAN

In [23]:
model = mmgan.GAN(image_size = size, hidden_dim = 400, z_dim = 100)
if torch.cuda.is_available():
    model = model.cuda()
trainer = mmgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 2e-4, D_lr = 2e-4, G_init = 5, D_steps = 1)

G pre-trained for 5 training steps.


/Users/heart/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


A Jupyter Widget

Epoch[1/1], G Loss: -0.3372, D Loss: 0.6338



# LS GAN

In [24]:
model = lsgan.GAN(image_size = size, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = lsgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)

A Jupyter Widget

Epoch[1/1], G Loss: 0.2025, D Loss: 0.1378



# DRA GAN

In [25]:
model = dragan.GAN(image_size = size, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = dragan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)

A Jupyter Widget

Epoch[1/1], G Loss: 0.7838, D Loss: 9.1333



# BE GAN

In [26]:
model = began.GAN(image_size = size, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = began.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = n, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)

A Jupyter Widget

Epoch[1/1], G Loss: 0.0644, D Loss: 0.8443, K: 0.0359, Convergence Measure: 1.0281

